# 구글 드라이브 마운트

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 라이브러리 설치

## kobert

In [ ]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-afqd9wrv
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-afqd9wrv
     |████████████████████████████████| 132 kB 4.3 MB/s 
     |████████████████████████████████| 344 kB 18.7 MB/s 
     |████████████████████████████████| 47.3 MB 1.1 MB/s 
     |████████████████████████████████| 4.5 MB 62.1 MB/s 
     |████████████████████████████████| 1.2 MB 75.0 MB/s 
     |████████████████████████████████| 3.5 MB 70.3 MB/s 
     |████████████████████████████████| 596 kB 73.9 MB/s 
     |████████████████████████████████| 6.5 MB 72.8 MB/s 
     |████████████████████████████████| 895 kB 81.6 MB/s 
     |████████████████████████████████| 67 kB 7.7 MB/s 
     |████████████████████████████████| 79 kB 11.5 MB/s 
     |████████████████████████████████| 8.6 MB 74.8 MB/s 
     |████████████████████████████████| 127 kB 68.2 MB/s 
  Created wheel for kobert: filename=ko

## PyKoSpacing 
공백제거 역할

In [ ]:
pip install git+https://github.com/haven-jeon/PyKoSpacing.git

  Cloning https://github.com/haven-jeon/PyKoSpacing.git to /tmp/pip-req-build-mgxsgwmx
  Running command git clone -q https://github.com/haven-jeon/PyKoSpacing.git /tmp/pip-req-build-mgxsgwmx
     |████████████████████████████████| 460.3 MB 7.3 kB/s 
     |████████████████████████████████| 132 kB 78.5 MB/s 
     |████████████████████████████████| 4.0 MB 76.3 MB/s 
     |████████████████████████████████| 1.2 MB 83.6 MB/s 
     |████████████████████████████████| 14.8 MB 1.1 MB/s 
     |████████████████████████████████| 462 kB 30.3 MB/s 
  Created wheel for pykospacing: filename=pykospacing-0.5-py3-none-any.whl size=2268580 sha256=eb5de99cca02dc3cbaf36d83db85d3f21269f5f5769f069e79793b4a838fb4e7
  Stored in directory: /tmp/pip-ephem-wheel-cache-pber3qso/wheels/9b/93/81/a2a7dc8c66ede5bf30634d20635f32b95eac7ca2ea8844058b
  Created wheel for wrapt: filename=wrapt-1.12.1-cp37-cp37m-linux_x86_64.whl size=68723 sha256=6b7fff0ee9f01b9950d7c677e125c7512d603a7cf3ab7e4881cac451b8b91c1f
  Stored in d

## py-hanspell
맞춤법 검사 역할

In [ ]:
pip install git+https://github.com/ssut/py-hanspell.git

  Cloning https://github.com/ssut/py-hanspell.git to /tmp/pip-req-build-atv7d5mq
  Running command git clone -q https://github.com/ssut/py-hanspell.git /tmp/pip-req-build-atv7d5mq
  Created wheel for py-hanspell: filename=py_hanspell-1.1-py3-none-any.whl size=4868 sha256=23345ec85bec7f90462b33aeaff444bc5641e478c030d1593745ada652828c3a
  Stored in directory: /tmp/pip-ephem-wheel-cache-j63rb9e9/wheels/ab/f5/7b/d4124bb329c905301baed80e2ae45aa14e824f62ebc3ec2cc4
Successfully built py-hanspell


## soynlp
중복제거 역할

In [ ]:
pip install soynlp

     |████████████████████████████████| 416 kB 4.3 MB/s 


# 라이브러리 임포트

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm.notebook import tqdm
import pandas as pd

# kobert 임포트
from kobert import get_tokenizer
from kobert import get_pytorch_kobert_model

## 옵티마이저 임포트
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

from soynlp.normalizer import *
import re

# CPU / GPU 사용 여부

In [ ]:
## CPU
#device = torch.device("cpu")

## GPU
device = torch.device("cuda:0")

# pytorch에서 bert, vocab 가져오기

In [ ]:
bertmodel, vocab = get_pytorch_kobert_model(cachedir=".cache")

/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


# 데이터 불러오기

## Emotion_multi_data

### 데이터 가져오기

In [241]:
emotion_multi_train = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/multi_train.xlsx')

### 사용하는 열 추출

In [242]:
emotion_multi_train = emotion_multi_train.loc[:,['사람문장1', '시스템응답1', '사람문장2', '시스템응답2', '사람문장3', '시스템응답3', '사람문장4', '시스템응답4','감정_대분류']]

### 공백이 추가된 라벨값 삭제

In [243]:
# 인덱스 추출
delete_multi_index = emotion_multi_train[emotion_multi_train['감정_대분류'].isin(['기쁨 ','불안 '])].index

# 삭제
emotion_multi_train.drop(delete_multi_index,axis=0,inplace=True)

### 결측값 공백으로 채우기

In [244]:
emotion_multi_train_fill = emotion_multi_train.fillna("")

### 라벨값 인코딩

In [245]:
emotion_multi_train_fill.loc[(emotion_multi_train_fill['감정_대분류'] == "분노"), '감정_대분류'] = 0  #분노 => 0
emotion_multi_train_fill.loc[(emotion_multi_train_fill['감정_대분류'] == "슬픔"), '감정_대분류'] = 1  #슬픔 => 1
emotion_multi_train_fill.loc[(emotion_multi_train_fill['감정_대분류'] == "상처"), '감정_대분류'] = 4  #상처 => 4
emotion_multi_train_fill.loc[(emotion_multi_train_fill['감정_대분류'] == "당황"), '감정_대분류'] = 5  #당황 => 5
emotion_multi_train_fill.loc[(emotion_multi_train_fill['감정_대분류'] == "불안"), '감정_대분류'] = 6  #불안 => 6
emotion_multi_train_fill.loc[(emotion_multi_train_fill['감정_대분류'] == "기쁨"), '감정_대분류'] = 7  #기쁨 => 7

### 각 문장을 연결한 구조 만들기

In [246]:
multi_result = []
for i in range(len(emotion_multi_train_fill)):
    data = ""
    for j in range(8):
        if emotion_multi_train_fill.iloc[i][j] == "":
            pass    
        else:
            data += emotion_multi_train_fill.iloc[i][j]
    
    multi_result.append([data, emotion_multi_train_fill.iloc[i][8]])

### csv 파일 저장

In [275]:
#emotion_multi_train = pd.DataFrame(np.c_[result, emotion_multi_fill["감정_대분류"]], columns=["sentence", "label"])
emotion_multi_train_fill.to_csv("test.csv", encoding="utf-8-sig", index=False)

## Emotion_sentence_data

### 데이터 불러오기

In [226]:
emotion_sentence_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/last_sentence.csv', encoding="utf-8-sig")

### 사용하는 열 추출

In [227]:
emotion_sentence_train = emotion_sentence_train.iloc[:, 1:]
emotion_sentence_train.columns = ["sentence", "label"]

### 결측값 삭제

emotion_sentence_train.dropna(inplace=True)

### 라벨값 오류 삭제

delete_sentence_index = emotion_sentence_train[emotion_sentence_train["label"].isin(['ㅍ','분','ㅈ중립','분ㄴ','중림','ㄴ중립','줄','공포'])].index
emotion_sentence_train.drop(delete_sentence_index,axis=0,inplace=True)

emotion_sentence_data_train_main.to_excel("test.xlsx", sheet_name='Sheet1', index=False)

### 라벨값 인코딩

emotion_sentence_train.loc[(emotion_sentence_train['label'] == "분노"), 'label'] = 0  #분노 => 0
emotion_sentence_train.loc[(emotion_sentence_train['label'] == "슬픔"), 'label'] = 1  #슬픔 => 1
emotion_sentence_train.loc[(emotion_sentence_train['label'] == "놀람"), 'label'] = 2  #놀람 => 2
emotion_sentence_train.loc[(emotion_sentence_train['label'] == "혐오"), 'label'] = 3  #혐오 => 3
emotion_sentence_train.loc[(emotion_sentence_train['label'] == "행복"), 'label'] = 8  #행복 => 8
emotion_sentence_train.loc[(emotion_sentence_train['label'] == "중립"), 'label'] = 9  #중립 => 9

### 띄어쓰기 검사

In [ ]:
#from pykospacing import Spacing
#spacing = Spacing()

#spacing_list = []

#for a in range(len(emotion_sentence_train)):
#    kospacing_sent = spacing(emotion_sentence_train.iloc[a][0])
#    spacing_list.append(kospacing_sent)

### 맞춤법 검사(stay)

from hanspell import spell_checker
fix_sentence = []
error_sentence = []

for i in emotion_sentence_train["sentence"]:
    check = spell_checker.check(i)
    word = ''

    for key, value in check.words.items():
        word += ' ' + key
    fix_sentence.append(word.lstrip())
    print(word)
fix_sentence

### 중복 제거
ㅋㅋㅋㅋㅋ -> ㅋㅋ

new1_list = []
for i in emotion_sentence_train['sentence']:
    repeat = repeat_normalize(i,num_repeats=2)
    new1_list.append(repeat)
emotion_sentence_train['remove_repeat'] = new1_list

### 특수문자 제거

new_list = []
for i in emotion_sentence_train['remove_repeat']:
    remove = re.sub(r"([?!])",r" \1 ",i)
    remove = re.sub(r"([^0-9a-zA-Z가-힣ㄱ-ㅎ!?. ])",'',remove)
    remove = remove.strip()
    new_list.append(remove)
emotion_sentence_train['remove_sign'] = new_list

emotion_sentence_train.drop(["sentence", "remove_repeat"], axis=1, inplace=True)

### bert데이터셋 리스트 만들기

In [ ]:
#result = []
#for a in range(len(emotion_sentence_train)):
#    result.append(emotion_sentence_train.iloc[a][1])

In [231]:
sentence_result = []

for q, label in zip(emotion_sentence_train['sentence'], emotion_sentence_train['label']):
    data = []
    data.append(q)
    data.append(label)

    sentence_result.append(data)

## sentence 단위 bert 데이터셋 전체 리스트

In [ ]:
#total_result = result + result1

### csv 파일 저장

sentence_result.to_csv("sentence.csv",encoding='utf-8-sig', index=False)

## Emotion_sentiment_data

### 디렉토리 파일 합치기

# 파일 리스트
dir_list = os.listdir('/content/drive/MyDrive/Colab Notebooks/Training/라벨링데이터_train')

# 디렉토리 경로
dir = '/content/drive/MyDrive/Colab Notebooks/Training/라벨링데이터_train/'

# 합칠 데이터프레임
emotion_sentiment_train = pd.DataFrame()

for i in dir_list:
    a = pd.read_csv(dir+i)
    emotion_sentiment_train = pd.concat([emotion_sentiment_train, a])

# 파일 리스트
dir_list = os.listdir('/content/drive/MyDrive/Colab Notebooks/Validation/라벨링데이터_validation')

# 디렉토리 경로
dir = '/content/drive/MyDrive/Colab Notebooks/Validation/라벨링데이터_validation/'

# 합칠 데이터프레임
emotion_sentiment_test = pd.DataFrame()

for i in dir_list:
    a = pd.read_csv(dir+i)
    emotion_sentiment_test = pd.concat([emotion_sentiment_test, a])

In [270]:
emotion_sentiment_total = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/last_sentiment.csv", encoding="utf-8-sig")

In [271]:
emotion_sentiment_total = emotion_sentiment_total.iloc[:, 1:]
emotion_sentiment_total.columns = ["sentence", "label"]

In [272]:
emotion_sentiment_total

,sentence,label
0,고속도로도 가깝고 안산역 좋죠,p
1,커피 너무 맛있네요. 저는 일회용 2개씩 넣는 거 좋더라고요,p
2,갓 구운 빵 냄새가 너무 좋아요,p
3,저는 이 제품이 더 마음에 드네요,p
4,음! 괜찮네요,p
...,...,...
21567,구매 상품 금액이 만원 이상일 경우 무료 배송이 오니 참고해 주시기 바랍니다.,m
21568,대리점이랑 통화하니 착지 변경해서 받을 수는 있는데 택배비가 4천 원이라 하네요 이...,m
21569,저희가 문의해 드려도 택배비는 같습니다.,m
21570,차가 두 대여서 개 구입하고픈데 그레이 브라운 가지 색 묶음 배송은 안 되나요 그리...,m


### 카테고리에 따라 데이터 균등 분할


label_list = emotion_sentiment_total["카테고리"].unique()

total_list = pd.DataFrame()

for i in label_list:
    positive_data = emotion_sentiment_total[(emotion_sentiment_total["카테고리"] == i) & (emotion_sentiment_total["감성"] == "p")]
    total_list = pd.concat([total_list, positive_data])

    negative_data = emotion_sentiment_total[(emotion_sentiment_total["카테고리"] == i) & (emotion_sentiment_total["감성"] == "n")].iloc[:300]
    total_list = pd.concat([total_list, negative_data])

    center_data = emotion_sentiment_total[(emotion_sentiment_total["카테고리"] == i) & (emotion_sentiment_total["감성"] == "m")].iloc[:1300]
    total_list = pd.concat([total_list, center_data])

### 사용하는 열 추출

total_list = total_list.loc[:, ["발화문", "감성"]]
total_list.columns = ["sentence", "sentiment"]

### 라벨값 인코딩

total_list.loc[(total_list['sentiment'] == "p"), 'sentiment'] = 0  # 긍정=> 0
total_list.loc[(total_list['sentiment'] == "n"), 'sentiment'] = 1  # 부정 => 1
total_list.loc[(total_list['sentiment'] == "m"), 'sentiment'] = 2  # 중립 => 2

### bert데이터셋 리스트 만들기

In [274]:
sentiment_list = []

for q, label in zip(emotion_sentiment_total['sentence'], emotion_sentiment_total['label']):
    data = []
    data.append(q)
    data.append(label)

    sentiment_list.append(data)

### csv 파일 저장

In [129]:
#sentiment_list.to_csv("sentiment.csv", encoding="utf-8-sig", index=False)

# 모델링 ===================

# kobert 토크나이저

한국어 모델을 공유합니다.

['▁한국', '어', '▁모델', '을', '▁공유', '합니다', '.']

In [ ]:
from kobert import get_tokenizer
from kobert import get_pytorch_kobert_model
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


# 데이터 셋 전처리

BERTSentenceTransform

bert모델에서 문장(input)을 Token Embeddings, Sentence Embedding, Transformer Positional Embedding 과정을 해주는 역할

In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

# 파라미터

In [ ]:
max_len = 40
# max_len = 80
# max_len = 360
batch_size = 32
warmup_ratio = 0.1
num_epochs = 3
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

# 데이터셋 만들기

### 감정데이터

In [247]:
data_train = BERTDataset(sentence_result, 0, 1, tok, 300, True, False)
data_train1 = BERTDataset(multi_result, 0, 1, tok, 300, True, False)

In [249]:
emotion_data = data_train + data_train1

### 감성데이터

In [134]:
sentiment_train = BERTDataset(data_list, 0, 1, tok, 40, True, False)

# 데이터 분할

In [ ]:
#result2 = result[:3000]
#result2 = result[30000:]

### 감정데이터

In [250]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(emotion_data, test_size=0.15, random_state=29)

In [251]:
train_dataloader = torch.utils.data.DataLoader(train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


### 감성데이터

In [136]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(sentiment_train, test_size=0.15, random_state=29)

In [137]:
train_dataloader = torch.utils.data.DataLoader(train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [252]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=10,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

# 모델 만들기

## 감정

In [253]:
emotion_model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)

##감성

In [200]:
sentiment_model = BERTClassifier(bertmodel, num_classes=3, dr_rate=0.5).to(device)

# 옵티마이저

## 감정

In [254]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in emotion_model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in emotion_model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

##감성

In [202]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in sentiment_model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in sentiment_model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [255]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [256]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [257]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [258]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

# 감정

In [259]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    emotion_model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(train_dataloader), total=len(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = emotion_model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(emotion_model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))

    emotion_model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(test_dataloader), total=len(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length = valid_length
        label = label.long().to(device)
        out = emotion_model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
        
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


  0%|          | 0/2697 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 2.25669002532959 train acc 0.15625


KeyboardInterrupt: ignored

In [ ]:
def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    emotion_model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = emotion_model(token_ids, valid_length, segment_ids)

        test_eval = []
        for i in out:
            logits = i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("긍정")
            elif np.argmax(logits) == 1:
                test_eval.append("부정")
            elif np.argmax(logits) == 2:
                test_eval.append("중립")
    

        print(">> 입력하신 내용에서 " + test_eval[0] + " 느껴집니다.")
        print(logits)

    print(torch.nn.functional.softmax(out, dim=1)[0])

# 감성

In [207]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    sentiment_model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(train_dataloader), total=len(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = sentiment_model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(sentiment_model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))

    sentiment_model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(test_dataloader), total=len(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length = valid_length
        label = label.long().to(device)
        out = sentiment_model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


  0%|          | 0/675 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 1.1981195211410522 train acc 0.3125
epoch 1 batch id 201 loss 0.059957507997751236 train acc 0.8523009950248757
epoch 1 batch id 401 loss 0.3130534589290619 train acc 0.895573566084788
epoch 1 batch id 601 loss 0.05068456754088402 train acc 0.9079658901830283
epoch 1 train acc 0.9106944444444445


  0%|          | 0/120 [00:00<?, ?it/s]

epoch 1 test acc 0.8861979166666667


...

epoch 2 test acc 0.8997395833333334


  0%|          | 0/675 [00:00<?, ?it/s]

epoch 3 batch id 1 loss 0.344520628452301 train acc 0.9375
epoch 3 batch id 201 loss 0.008040186949074268 train acc 0.9690609452736318
epoch 3 batch id 401 loss 0.3497650623321533 train acc 0.9732699501246883
epoch 3 batch id 601 loss 0.007955347187817097 train acc 0.9748856073211315
epoch 3 train acc 0.9751388888888889


  0%|          | 0/120 [00:00<?, ?it/s]

epoch 3 test acc 0.9020833333333333


In [217]:
def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, 150, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    print(another_test[0])
    sentiment_model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = sentiment_model(token_ids, valid_length, segment_ids)

        test_eval = []
        for i in out:
            logits = i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("긍정")
            elif np.argmax(logits) == 1:
                test_eval.append("부정")
            elif np.argmax(logits) == 2:
                test_eval.append("중립")
    

        print(">> 입력하신 내용에서 " + test_eval[0] + " 느껴집니다.")
        print(logits)

    print(torch.nn.functional.softmax(out, dim=1)[0])

In [218]:
out = predict("물건이 마음에 드네요. 다행이네요. 근데 버려버릴꺼예요. 마음에 안들어졌어요. 무슨 이런걸 파는거죠?")

(array([   2, 2137, 7096, 1917, 6896, 1788, 5703,  517,   54, 1562, 7881,
       7096, 5703,  517,   54, 1221, 5850, 2302, 6060, 6323, 6135, 5604,
       6957, 6999,  517,   54, 1917, 6896, 3135, 5934, 7248, 6857,  517,
         54, 2110, 3672, 5390, 4799, 5760, 5377, 7275,  633,    3,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1], 

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


>> 입력하신 내용에서 긍정 느껴집니다.
[ 3.7455895 -0.7504973 -2.7477758]
tensor([0.9875, 0.0110, 0.0015], device='cuda:0', grad_fn=<SelectBackward0>)


In [193]:
model_save_name = 'sentiment.pt'
path = F"/content/drive/MyDrive/{model_save_name}" 
torch.save(model.state_dict(), path)

In [ ]:
model_save_name = 'classifier.pt'
path = F"/content/drive/MyDrive/{model_save_name}"
model.load_state_dict(torch.load(path))

<All keys matched successfully>